# Mise en place du deuxième algorithme de recherche de chemins sur les premiers exemples de vésicules


## <ins>Importation et construction des tableaux de réaction</ins>

##### Importation des bibliothéques définis par l'utilisateur

In [1]:
#bibliothéque python
import re
import time
import pandas as pd
import numpy as np

#importation code perso
import recherche_chemin
import algoresolution_système
from creation_CRN_v2 import *
from brenda import read_data

def numero(texte):
    if type(texte) is str:
        return recherche_chemin.numero(texte,listeNomsMolecules)
    if type(texte) is list:
        return recherche_chemin.numero2(texte,listeNomsMolecules)

##### Fonctions de lecture du fichier catalogue ou de brenda

In [2]:
def get_data_from_file(file):
    elmts = []
    inputs = []
    enzymes = []
    regex = ["(present\()|(, [e\-0-9]+\))|\.", "(MA.*for )|\+|(=>)|\."]
    blocs = {"inputs": (regex[0], inputs), "enzymes": (regex[0], enzymes), "elmts": (regex[1], elmts)}

    """ Création du tableau "reaction" de l'ensemble des réactions possible données par le fichier file"""
    friends = {'test': []}  # Chaque elmt et l'ensemble des elements avec lesquels il reagit
    # par contre les enzymes ne sont pas comptés comme éléments, du coup ils ne font pas partie des clés du dict.

    reaction = []  # Liste des réactions en version texte
    with open(file) as f:
        while f.readline() != "% Inputs\n":
            continue
        bloc = 'inputs'
        while True:
            l = f.readline()
            if not l:
                break

            if l[0] == '%':
                if l.find("Enzymes") > 0:
                    bloc = 'enzymes'
                    continue
                if l.find('reaction') > 0:
                    bloc = 'elmts'
                    continue
                continue

            if len(l) < 3:
                continue

            if bloc == 'elmts':
                elmt = l.split('=>')
                reactifs = re.sub("(MA.*for )|\+", " ", elmt[0]).split()
                for r in reactifs:
                    try:
                        friends[r].extend([e for e in reactifs if e not in friends[r]])
                    except:
                        friends.update({r: [e for e in reactifs if e != r]})
                produits = re.sub("\+|\.", " ", elmt[1]).split()
                reaction.append((reactifs, produits))
                elts = reactifs + produits
            else:
                elts = re.sub(blocs[bloc][0], '', l).split()

            blocs[bloc][1].extend([e for e in elts if (e not in blocs[bloc][1])])

    return elmts, enzymes, reaction

def base_cataloge():
    # Utilisation du catalogue
    file = "catalog.bc"
    enzymes, elmts, reaction = get_data_from_file(file)
    listeNomsMolecules = enzymes + elmts
    listeNomsMolecules.pop(26)

    listeReactionsBrut = []  # Liste des réactions en version numéros
    for a in reaction:
        reac = ([], [])
        for m in a[0]:
            if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
                reac[0].append(recherche_chemin.numero('H2O2',listeNomsMolecules))
            else:
                reac[0].append(recherche_chemin.numero(m,listeNomsMolecules))
        for m in a[1]:
            if m == 'H_20_2' or m == 'H202' or m == 'H_2O_2':
                reac[1].append(recherche_chemin.numero('H2O2',listeNomsMolecules))
            else:
                reac[1].append(recherche_chemin.numero(m,listeNomsMolecules))
        listeReactionsBrut.append(reac)
    # Fin  preprocessing du Fichier
            
    return listeReactionsBrut,listeNomsMolecules

def base_brenda():
    # Utilisation de Brenda
    print("Start read data")
    data = read_data('data.json')
    listeReactionsBrut = data['reactions'] # [ [[liste des reactifs],[liste des produits],[kms],[kcats],[phs]] autant de fois qu'il y a de réactions]
    listeNomsMolecules = data['molecules_list'] # inclus molecules et enzymes
    dictionnaireNomsMolecules = data['dic_name_to_idx'] # Pour avoir l'indice à partir du nom
    #listeReactionParMolecules = data['reactions_per_molecule']
    print("End read data")
    return listeReactionsBrut,listeNomsMolecules,dictionnaireNomsMolecules

##### Création du tableau de réactions

In [3]:
"""    Déclaration tableaux de réactions et molécules"""
listeReactionsBrut,listeNomsMolecules = base_cataloge()
#listeReactionsBrut,listeNomsMolecules,dictionnaireNomsMolecules = base_brenda()

listeReactionParMolecules = [[] for k in range(len(listeNomsMolecules))]
for numReaction, reaction in enumerate(listeReactionsBrut):
    for reactif in reaction[0]:
        listeReactionParMolecules[reactif].append(numReaction)

## <ins>Résolution d'exemples</ins>

#### <ins>Test 1</ins>
glucose et acetone donnent resorufin. Avec le catalogue

In [4]:
##glucose et acetone donnent resorufin
start_time = time.time()

EspecesInitiales=numero(['AO', 'ADH', 'G_1DH', 'NAD', 'resazurin', 'HRP', 'H2O2'])

nbEtapeMax=15

A=numero('acetoneext')
B=numero('glucoseext')
C=numero('resorufin')

MelangesInitiaux,res=déterminationCRN (A,B,C,nbEtapeMax,"ET",EspecesInitiales,listeReactionsBrut,listeReactionParMolecules)

#Affichage des résultats
affichage_mélanges_i(MelangesInitiaux,C,listeNomsMolecules)
print()
print("Affichage du CRN trouvé")
affichage_CRN(res[1],listeReactionsBrut,listeReactionParMolecules,listeNomsMolecules)

print("Temps de calcul : ",time.time()-start_time)
print()
print('------------------------------------------------------------------')
print()

Pour obtenir la molécule  resorufin
   En 3 étape avec le mélange : HRP,resazurin,H2O2
   En 12 étape avec le mélange : glucoseext,acetoneext,HRP,resazurin,AO,NAD,G_1DH,H2O2,ADH
   En 12 étape avec le mélange : glucoseext,acetoneext,HRP,resazurin,AO,NAD,G_1DH,ADH

Affichage du CRN trouvé
   En 12 étape avec le mélange initial : glucoseext, acetoneext, HRP, resazurin, AO, NAD, G_1DH, ADH

   Réactions du CRN :
glucoseext -> glucose
acetoneext -> acetone
NAD + G_1DH -> CCia1
NAD + ADH -> DDib1
glucose + CCia1 -> CCib1
glucose + Cia1 -> Cib1
CCia1 -> G_1DH + NAD
Cia1 -> G_1DH + NAD
DDib1 -> ADH + NAD
CCib1 -> CCia1 + glucose
CCib1 -> CCfa1 + NADH
Cib1 -> Cia1 + glucose
Cib1 -> Cfa1 + NADH
CCfa1 -> G_1DH + gluconolacrone
NADH + ADH -> CCia2
Cfa1 -> G_1DH + gluconolacrone
CCia2 -> ADH + NADH
CCia2 + acetone -> CCib2
CCib2 -> CCia2 + acetone
CCib2 -> CCfa2 + NAD
CCfa2 -> ADH + isopropanol
isopropanol + AO -> CCf3
CCf3 -> AO + isopropanol
CCf3 -> CCio3 + H2O2
CCio3 -> AO + HRP2
H2O2 + HRP -> 

#### <ins>Test 2</ins>
NO et glucose donnent gluconolacrone. Avec le catalogue

In [5]:
##NO et glucose donnent gluconolacrone
start_time = time.time()

EspecesInitiales=numero(['H2O2','acetoneext', 'ABTS', 'ADH', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'NAD'])
nbEtapeMax=50
A=numero('NO2')
B=numero('glucoseext')
C=numero('DAFF')

MelangesInitiaux,res=déterminationCRN (A,B,C,nbEtapeMax,'ET',EspecesInitiales,listeReactionsBrut,listeReactionParMolecules)

#Affichage des résultats
affichage_mélanges_i(MelangesInitiaux,C,listeNomsMolecules)
print()
print("Affichage du CRN trouvé")
affichage_CRN(res[1],listeReactionsBrut,listeReactionParMolecules,listeNomsMolecules)

print("Temps de calcul : ",time.time()-start_time)
print()
print('------------------------------------------------------------------')
print()

Pour obtenir la molécule  DAFF
   En 11 étape avec le mélange : glucoseext,NO2,NAD,G_1DH,O2,DAF,NR
   En 11 étape avec le mélange : glucoseext,NO2,HRP2,NAD,G_1DH,O2,DAF,NR
   En 11 étape avec le mélange : glucoseext,NO2,HRP2,NAD,G_1DH,O2,DAF,NR,ADH
   En 11 étape avec le mélange : glucoseext,NO2,HRP2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD
   En 11 étape avec le mélange : glucoseext,NO2,NAD,G_1DH,O2,DAF,NR,ADH
   En 11 étape avec le mélange : glucoseext,NO2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD,ADH
   En 11 étape avec le mélange : glucoseext,NO2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD
   En 11 étape avec le mélange : glucoseext,NO2,HRP2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD,ADH
   En 12 étape avec le mélange : glucoseext,NO2,HRP,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD
   En 12 étape avec le mélange : glucoseext,NO2,HRP,HRP2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD
   En 12 étape avec le mélange : glucoseext,NO2,HRP,HRP2,NAD,G_1DH,O2,DAF,NR,ABTS,H2O2,POD,ADH
   En 12 étape avec le mélange : glucoseext,NO2,HRP,NAD,G_1DH

#### <ins>Test 3</ins>
Lactateext ET EtOHext donnent ABTSOX. Avec le catalogue

In [6]:
## Lactateext ET EtOHext donnent ABTSOX
start_time = time.time()
    
EspecesInitiales=numero(['ABTS', 'ADH', 'NAD', 'resazurin', 'HRP', 'AO', 'HRP2', 'POD', 'NR', 'G_1DH', 'O2', 'DAF', 'LO'])
nbEtapeMax=50
A=numero('Lactateext')
B=numero('EtOHext')
C=numero('ABTSOX')

MelangesInitiaux,res=déterminationCRN (A,B,C,nbEtapeMax,'ET',EspecesInitiales,listeReactionsBrut,listeReactionParMolecules)

#Affichage des résultats
affichage_mélanges_i(MelangesInitiaux,C,listeNomsMolecules)
print()
print("Affichage du CRN trouvé")
affichage_CRN(MelangesInitiaux[C][0],listeReactionsBrut,listeReactionParMolecules,listeNomsMolecules)

print("Temps de calcul : ",time.time()-start_time)
print()
print('------------------------------------------------------------------')
print()

Pour obtenir la molécule  ABTSOX
   En 5 étape avec le mélange : Lactateext,ABTS,LO,POD
   En 7 étape avec le mélange : Lactateext,EtOHext,NAD,G_1DH,ABTS,LO,POD,ADH
   En 7 étape avec le mélange : Lactateext,EtOHext,NAD,ABTS,LO,POD,ADH
   En 7 étape avec le mélange : Lactateext,HRP,ABTS,LO,POD
   En 8 étape avec le mélange : Lactateext,EtOHext,HRP2,NAD,G_1DH,ABTS,LO,POD,ADH
   En 8 étape avec le mélange : Lactateext,EtOHext,HRP2,NAD,ABTS,LO,POD,ADH
   En 8 étape avec le mélange : Lactateext,EtOHext,HRP2,NAD,NR,ABTS,LO,POD,ADH
   En 8 étape avec le mélange : Lactateext,EtOHext,NAD,G_1DH,NR,ABTS,LO,POD,ADH
   En 8 étape avec le mélange : Lactateext,EtOHext,NAD,NR,ABTS,LO,POD,ADH
   En 9 étape avec le mélange : Lactateext,EtOHext,HRP2,NAD,G_1DH,NR,ABTS,LO,POD,ADH
   En 9 étape avec le mélange : Lactateext,EtOHext,HRP,HRP2,NAD,G_1DH,ABTS,LO,POD,ADH
   En 9 étape avec le mélange : Lactateext,EtOHext,HRP,HRP2,NAD,G_1DH,NR,ABTS,LO,POD,ADH
   En 9 étape avec le mélange : Lactateext,EtOHext,HRP